### This notebook provides code for embeddings visualization with tensorboard. Embeddings are computed from a Vision Transformers Masked Auto Encoder neural network, that was trained over Imagenet with image size 64 x 64. Embeddings can be visualized through PCA/T-SNE/UMAP projection from tensorboard framework, and we also provide code to visualize image content and labels with tensorboard. 

In this notebook we compute embeddings of weather image recognition dataset available <a href="https://www.kaggle.com/datasets/fceb22ab5e1d5288200c0f3016ccd626276983ca1fe8705ae2c32f7064d719de">here<a> and holding CC0 licence.




# Imports 

In [1]:
from utils.projector_utils import DATA, images_to_sprite,extractor,path_labels_extractor
from utils.load_models import load_mae
from utils.shape_filtering import shape_filter
import os 
from tqdm import tqdm
import random
import cv2
import csv
inet_mean = [0.485, 0.456, 0.406]
inet_std = [0.229, 0.224, 0.225]
import numpy as np

# Creating logs folders for tensorboard logs and data


In [2]:

current = os.getcwd()
DATA_PATH = current + '\\dataset' #folder to store images
LOG_DIR = current + '\\logs'

if not os.path.exists(LOG_DIR):
    os.mkdir(LOG_DIR)

if not os.path.exists(DATA_PATH): #extract images if needed
    from zipfile import ZipFile
    with ZipFile("archive.zip", 'r') as zObject: #the weather image recognition dataset provide this "archive.zip" file when it's downloaded from kaggle.
        zObject.extractall()

### Check that cuda is available to make things faster and load model

In [3]:
import torch
device = 'cuda' if torch.cuda.is_available() else('cpu')
print(device)
#Loading VitMAE model pretrained on imagenet 64*64
model = load_mae().to(device)

cuda
Model loaded.


# Load data

In [4]:

#Loading list of image files
imgs_list,imgs_labels,_,n_bads = path_labels_extractor(DATA_PATH)


100%|██████████| 11/11 [00:27<00:00,  2.50s/it]


In [5]:
imgs_list = (np.array((imgs_list,imgs_labels)).T).tolist() #we need list of typle for this script

In [6]:
#Tensorboard won't be able to display more than 1500 images, so it is recommended to sample some of them for visualization. Here we sample among the 5 first classes.
imgs_list = random.sample(imgs_list[:2662],1500)

In [7]:
imgs_list = shape_filter(imgs_list) #filtering out image with wrong number of channels
weather = DATA(imgs_list, transform = None, mean = inet_mean,std = inet_std,img_size = 64)
e = extractor(model,weather,device) #extractor is used to extract images, embeddings or label

100%|██████████| 1500/1500 [00:08<00:00, 187.05it/s]

Number of bad images :  0  /  1500


# Generate embeddings and sprites, might take some time depending on your GPU

In [8]:
images_pil = []
images_embeddings = []
labels = []
for x in tqdm(range(weather.__len__())): 
    img_pil = e.get_img(x)
    img_embedding = e.get_embed(x)
    images_embeddings.append(img_embedding.cpu().detach().numpy())
    images_pil.append(np.array(img_pil))
    # Assuming your output data is directly the label
    label = e.get_label(x)
    labels.append(label)

100%|██████████| 1500/1500 [00:33<00:00, 44.86it/s]


In [9]:
#Saving files for tensorboard
sprite = images_to_sprite(np.array(images_pil))
cv2.imwrite(f'{LOG_DIR}\\sprite.jpg', sprite)

with open(f'{LOG_DIR}\\feature_vecs.tsv', 'w') as fw:
    csv_writer = csv.writer(fw, delimiter='\t')
    csv_writer.writerows(images_embeddings)
    
with open(f'{LOG_DIR}\\metadata.tsv', 'w') as file: 
    for label in labels:
        #file.write(f"{classes[str(label)]}\n")
        file.write(f"{label}\n")    

#to get number of images per line in the sprite
int(np.ceil(np.sqrt(np.array(images_pil).shape[0]))) 

39

# Build proper config file for tensorboard and then visualize the embeddings


In [10]:
"""
This is what the config file will look like.

embeddings {
  metadata_path: "metadata.tsv"
  tensor_path: "feature_vecs.tsv"
  sprite {
    image_path: "sprite.jpg"
    single_image_dim: 64
    single_image_dim: 64
  }
}
"""

text_file = open(LOG_DIR+'\\projector_config.pbtxt', "w")
text_file.write('embeddings {\n  metadata_path: "metadata.tsv"\n  tensor_path: "feature_vecs.tsv"\n  sprite {\n    image_path: "sprite.jpg"\n    single_image_dim: 64\n    single_image_dim: 64\n  }\n}')
text_file.close()

In [11]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass

%load_ext tensorboard

In [15]:
#%reload_ext tensorboard #In case you made some experiments and whant to display another tensorboard.

In [ ]:
%tensorboard --logdir ./logs #Go to projector

In [17]:
#On Windows, you might not be able to kill tensorboard process. In this case you should clear its data by removing this fodler : C:\Users\USER\AppData\Local\Temp\.tensorboard-info